In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# import seaborn as sns
import os
import re
from scipy.optimize import curve_fit
# sns.set_theme(style = "whitegrid")

In [ ]:
matplotlib.matplotlib_fname()

In [ ]:
with open("net-size-guantlet-ratings.txt") as f:
    text = f.readlines()

In [ ]:
data = [s.split()[1:5] for s in text]
names, _, elos, errors = zip(*data[2:])
names = list(map(int, names))
elos = list(map(float, elos))
errors = list(map(float, errors))
up_down_errors = [errors, errors]

In [ ]:
# Create plot
fig, ax = plt.subplots()

# Customize plot
ax.set_ylabel("Elo")
ax.set_yticks(list(range(0, 550, 50)))
ax.set_xlabel("Number of Neurons")
ax.set_xscale("log", base = 2)
ax.set_xticks([2 ** (x / 2) for x in range(4 * 2, 14 * 2)])
ax.set_xticklabels([2 ** (x // 2) if x % 2 == 0 else None for x in range(4 * 2, 14 * 2)])
ax.set_xlim(16, 8192)
ax.set_title("Elo vs. Net Size")
(_, caps, _) = ax.errorbar(
    names,
    elos,
    yerr=up_down_errors,
    fmt='o',
    label="scaling",
    # add line through the data
    ls='--',
    lw=1.5,
)
for cap in caps:
    cap.set_markeredgewidth(1.0)
# ax.plot([1536], [598.0], "x", label="master")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Data points
WDL = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
values = np.array([21.8, -3.7, 22.5, 3.1, 13.0, 0.0, -4.9, -25.9, -10.0, -37.8, -46.3])
errors = np.array([15.6, 15.7, 15.5, 16.0, 15.7, 0.0, 15.4, 15.6, 15.4, 15.0, 15.7])
lower_bounds = np.subtract(values, errors)
upper_bounds = np.add(values, errors)

fig, ax = plt.subplots()

# Plotting data with error bars
ax.errorbar(WDL, values, yerr=errors, fmt='o', label='Elo')

# Define fitting functions
def linear_fit(x, a, b):
    return a * x + b

def quadratic_fit(x, a, b, c):
    return a * x**2 + b * x + c

def cubic_fit(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

# Fit the data using the defined functions
params_linear, _ = curve_fit(linear_fit, WDL, values)
params_quadratic, _ = curve_fit(quadratic_fit, WDL, values)

lb_params_quadratic, _ = curve_fit(quadratic_fit, WDL, lower_bounds)
ub_params_quadratic, _ = curve_fit(quadratic_fit, WDL, upper_bounds)

# Generate x values for plotting the fit curves
x_fit = np.linspace(0, 1, 100)

# Plot the fitted curves
# ax.plot(x_fit, linear_fit(x_fit, *params_linear), label='Linear Fit')
# ax.plot(x_fit, quadratic_fit(x_fit, *params_quadratic), label='Quadratic Fit')

ax.plot(x_fit, quadratic_fit(x_fit, *ub_params_quadratic), label='Upper Bound')
ax.plot(x_fit, quadratic_fit(x_fit, *lb_params_quadratic), label='Lower Bound')

# Plot settings
ax.set_xlabel('WDL-λ')
ax.set_ylabel('Elo')
ax.set_title('Elo of different WDL-λ values')
ax.set_xticks(np.arange(0, 1.1, 0.1))
ax.legend()
ax.grid(True)
fig.tight_layout()

# Show the plot
plt.show()

# Display the parameters of the fitted curves
params_linear, params_quadratic, ub_params_quadratic, lb_params_quadratic